In [7]:
import torch
import torch.nn as nn
import string
import random
import unidecode
from torch.utils.tensorboard import SummaryWriter

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_characters = string.printable
n_characters = len(all_characters)

file = unidecode.unidecode(open('/kaggle/input/names-txt/names.txt').read())
file[0:20]

'emma\nolivia\nava\nisab'

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size) #học embedding của text đưa vào
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

In [19]:
class Generator:
    def __init__(self):
        self.chunk_len = 250
        self.num_epochs = 500
        self.batch_size = 1
        self.print_every = 50
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003

    def char_tensor(self, string): #lấy kí tự rồi biến thành vector
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c]) #tensor[c] = index của string[c] trong dãy printable
        return tensor

    def get_random_batch(self):
        start_idx = random.randint(0, len(file) - self.chunk_len) #để đảm bảo batch luôn đủ chunk_len mẫu
        end_idx = start_idx + self.chunk_len + 1
        text_str = file[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)

        for i in range(self.batch_size):
            text_input[i, :] = self.char_tensor(text_str[:-1]) #lấy từ đầu đến vị trí trước cuối cùng của text_str sau đó biến thành vector
            text_target[i, :] = self.char_tensor(text_str[1:]) #Lấy từ vị trí 1 (0 based index) đến cuối cùng của text_str sau đó biến thành vector

        return text_input.long(), text_target.long()

    def generate(self, initial_str='Ab', predict_len=100, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        predicted = initial_str

        for p in range(len(initial_str) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)
        last_char = initial_input[-1]

        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)

        return predicted

    def train(self):
        self.rnn = RNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device) #input_size = output_size = chunk_len
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(1, self.num_epochs + 1):
            input, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            optimizer.zero_grad()
            loss = 0
            input = input.to(device) #tensor or model sẽ có .to(device)
            target = target.to(device)

            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(input[:, c], hidden, cell) #bản chất là gọi self.rnn.forward(), mà forward đã unsqueeze shape rồi
                loss += criterion(output, target[:, c])                      #nên ta vẫn để input là tensor 2 chiều
            loss.backward()
            optimizer.step()
            loss += loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f'Loss: {loss}')
                print(self.generate())

In [20]:
generator = Generator()
generator.train()

Loss: 652.9052734375
Abk
elovhy
nolre
anabe
tmabaaa
lkens
aav
nrpezn
rrann
dein
riarae
ayru
hlenoneno
rloohi
naanynna
krlan
Loss: 588.9419555664062
Aba
yansi
mtore
nhoimmom
ganiizl
toam
anon
namifnah
irhila
sapanann
doer
nongom
arin
hylake
dhdinte
am
Loss: 589.1083984375
Abi
cali
yelwee
jeell
lasie
le
ahaze
jelia
dorbg
stiiri
xellla
ilisa
fer
soiyne
zoshtey
atel
sil
keren
Loss: 511.633056640625
Abee
avron
haren
ccama
cisien
aamie
kainmanah
sunltay
geradam
emlah
selistr
gellevan
measy
mann
caysai
Loss: 544.9044189453125
Abnaive
bherlien
keret
merariele
tyeylionn
tollia
arili
eoli
zavini
jader
eleeri
aza
baire
asean
romon
Loss: 557.6331787109375
Aba
uile
yindon
joyah
cheylyan
lalynne
cyelie
morley
komie
gunlee
maqinen
tyani
edkani
kamon
bufon
alv
Loss: 611.7931518554688
Abea
bavid
jazaodh
saston
dokele
qeni
ailia
falee
selona
reyl
amina
aleah
saidela
aydrinn
ajae
amheley
Loss: 564.8148803710938
Abiste
asonisa
warat
shaysion
tamdrithary
mibran
mazix
racieh
whyssenin
deydelleia
brisser
deo

In [55]:
generator = Generator()
input, target = generator.get_random_batch()

In [67]:
input, target

(tensor([[96, 20, 27, 18, 28, 29, 34, 96, 21, 10, 14, 21, 96, 21, 14, 18, 16, 17,
          96, 21, 18, 31, 14, 21, 34, 96, 21, 18, 35, 14, 29, 29, 14, 96, 22, 10,
          17, 10, 21, 18, 10, 96, 22, 18, 27, 10, 11, 14, 21, 96, 23, 10, 18, 24,
          22, 18, 96, 23, 10, 21, 18, 34, 10, 17, 96, 25, 10, 34, 28, 24, 23, 96,
          25, 27, 14, 28, 21, 18, 14, 96, 27, 17, 18, 21, 14, 34, 96, 27, 18, 31,
          14, 27, 21, 34, 23, 96, 27, 24, 28, 10, 21, 14, 14, 23, 96, 27, 30, 23,
          10, 96, 29, 10, 29, 34, 10, 23, 10, 96, 29, 18, 14, 27, 27, 10, 96, 31,
          10, 23, 34, 10, 96, 10, 11, 11, 34, 16, 10, 18, 21, 96, 10, 13, 10, 22,
          10, 27, 18, 28, 96, 10, 21, 18, 23, 10, 17, 96, 10, 21, 24, 23, 18, 96,
          10, 23, 10, 14, 21, 21, 14, 96, 10, 23, 16, 14, 21, 18, 14, 96, 10, 23,
          19, 10, 96, 11, 10, 21, 14, 18, 16, 17, 96, 11, 10, 34, 10, 23, 96, 11,
          14, 29, 17, 96, 11, 21, 18, 28, 28, 96, 11, 30, 28, 17, 27, 10, 96, 12,
          10, 21

In [71]:
print(input.shape, target.shape)

torch.Size([1, 250]) torch.Size([1, 250])


In [85]:
a = torch.tensor([[[1,2,3], [1,2,3]]]).view(-1)
a.shape

torch.Size([6])